# Fine-Tuning Llama 2

In [ ]:
# %capture
# %pip install accelerate peft bitsandbytes transformers trl

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
# Model from Hugging Face hub
base_model = "meta-llama/Llama-2-7b-hf"

# New instruction dataset
dataset = "code.csv"

# Fine-tuned model
new_model = "Llama-2-7b-hf-simple-code"

In [ ]:
#Importing the dataset
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split

# Step 1: Load your dataset
data = pd.read_csv("code.csv", delimiter=";")  # Adjust delimiter if necessary

# Step 2: Split the dataset into train and test sets
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

# Step 3: Save the train and test sets to CSV files
train_df.to_csv("train.csv", index=False)
test_df.to_csv("test.csv", index=False)

# Step 4: Load the datasets in Hugging Face format
train_dataset = load_dataset("csv", data_files={"train": "train.csv"})["train"]
test_dataset = load_dataset("csv", data_files={"test": "test.csv"})["test"]

instruction = """You are a programming expert specialized in reasoning and executing python algorithms. The user will provide you with a python code snippet and ask you to execute it with the given input. You should execute the code and provide the output to the user. If the code contains an error, you should identify the error and provide the user with a hint to fix it. If the code is correct, you should provide only the code output to the user."""

def format_chat_template(row):

    row_json = [{"role": "system", "content": instruction },
               {"role": "user", "content": row["python_code"]+ "\n "+ row["execution_example"]},
               {"role": "assistant", "content": row["execution_result"]}]
    
    # row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

# Load CSV files for train and test datasets separately
train_dataset = load_dataset('csv', data_files='train.csv', header=None)
test_dataset = load_dataset('csv', data_files='test.csv', header=None)

# Manually assign column names for both train and test datasets
train_dataset = train_dataset['train'].rename_column('0', 'python_code')
train_dataset = train_dataset.rename_column('3', 'human_reasoning')
train_dataset = train_dataset.rename_column('1', 'execution_example')
train_dataset = train_dataset.rename_column('2', 'execution_result')


test_dataset = test_dataset['train'].rename_column('0', 'python_code')
test_dataset = test_dataset.rename_column('3', 'human_reasoning')
test_dataset = test_dataset.rename_column('1', 'execution_example')
test_dataset = test_dataset.rename_column('2', 'execution_result')

train_dataset = train_dataset.map(format_chat_template, num_proc= 4)
test_dataset = test_dataset.map(format_chat_template, num_proc= 4)


In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

In [ ]:
from tensorboard import notebook
log_dir = "results/runs"
notebook.start("--logdir {} --port 4000".format(log_dir))
